<img style="float: right;" src="hyperstream.svg">

# HyperStream Tutorial 5: Workflows

- What are workflows?
- 

In [1]:
%load_ext watermark

import sys
sys.path.append("../") # Add parent dir in the Path

from hyperstream import HyperStream
from hyperstream import TimeInterval
from hyperstream.utils import UTC

from datetime import datetime

%watermark -v -m -p hyperstream -g

hs = HyperStream(loglevel=20)
print hs

CPython 2.7.6
IPython 5.3.0

hyperstream 0.3.0-beta

compiler   : GCC 4.8.4
system     : Linux
release    : 3.19.0-80-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
Git hash   : 33621000d8f335505eac32a5a274194d2588a4c9
HyperStream version 0.3.0-beta, connected to mongodb://localhost:27017/hyperstream


In [2]:
reader_tool = hs.plugins.example.tools.csv_reader('data/sea_ice.csv')
sea_ice_stream = hs.channel_manager.memory.get_or_create_stream("sea_ice")
ti = TimeInterval(datetime(1990, 1, 1).replace(tzinfo=UTC), datetime(2012, 1, 1).replace(tzinfo=UTC))

reader_tool.execute(sources=[], sink=sea_ice_stream, interval=ti)
for key, value in sea_ice_stream.window().items()[:10]:
    print '[%s]: %s' % (key, value)

[1990-02-01 00:00:00+00:00]: [13.33, 2.15]
[1990-03-01 00:00:00+00:00]: [13.44, 2.71]
[1990-04-01 00:00:00+00:00]: [12.16, 5.1]
[1990-05-01 00:00:00+00:00]: [10.84, 7.37]
[1990-06-01 00:00:00+00:00]: [9.12, 10.26]
[1990-07-01 00:00:00+00:00]: [6.44, 12.17]
[1990-08-01 00:00:00+00:00]: [4.92, 13.95]
[1990-09-01 00:00:00+00:00]: [4.5, 14.3]
[1990-10-01 00:00:00+00:00]: [6.67, 13.71]
[1990-11-01 00:00:00+00:00]: [9.58, 11.24]


In [4]:
from hyperstream import Tool

from utils import plot_high_chart

class ThresholdDetector(Tool):
    def __init__(self):
        super(ThresholdDetector, self).__init__()
    def _execute(self, sources, alignment_stream, interval):
        my_time, my_data = zip(*[(key.__str__(), value[0]) for key, value in sources[0].window(interval, force_calculation=True).items()])

        plot_high_chart(my_time, my_data, type="high_stock")
        return 

In [5]:
from hyperstream import StreamId

M = hs.channel_manager.memory
threshold_detection = M.get_or_create_stream(stream_id=StreamId("threshold_detection"))

threshold_tool = ThresholdDetector()
threshold_tool.execute(sources=[sea_ice_stream], sink=threshold_detection,
                       interval=ti, alignment_stream=None)

TypeError: 'NoneType' object is not iterable

In [6]:
M = hs.channel_manager.memory

sea_levels = M.get_or_create_stream(stream_id=StreamId("sea_levels"), meta_data=((""))

SyntaxError: invalid syntax (<ipython-input-6-e5a94320db46>, line 3)

In [7]:
w = hs.create_workflow(workflow_id="sea_ice_level_detector",
                       name="Threshold deterctor",
                       description="Sleep detection using the van Hees algorithm",
                       owner="WP5",
                       online=False)

In [8]:
S = hs.channel_manager.sphere
M = hs.channel_manager.memory
T = hs.channel_manager.tools
D = hs.channel_manager.mongo

AttributeError: 'ChannelManager' object has no attribute 'sphere'

In [9]:
w.create_node(channel="accel", channel=S, ["H"])

SyntaxError: keyword argument repeated (<ipython-input-9-cec22c770a4e>, line 1)

In [ ]:
w.execute(time_interval) 